In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
seir_data = pd.read_csv(
    "./csvs/model_4_2.csv",
    index_col=0,
)
config = {
    "tot_popn": 5234.,
    "infous_0": 2.,
    "end_time": 200.,
    "t_step": 0.5,
    "prop_immune_0": 0.3,
}
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
model = CompartmentalModel(
    times=(0, config["end_time"]),
    compartments=compartments,
    infectious_compartments=["Infectious"],
    timestep=config["t_step"],
)
n_immune_0 = config["prop_immune_0"] * config["tot_popn"]
model.set_initial_population(
    distribution={
        "Susceptible": config["tot_popn"] - config["infous_0"] - n_immune_0, 
        "Infectious": config["infous_0"],
        "Immune": n_immune_0,
    }
)

# Transitions
ave_infous = Parameter("ave_infous")
model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("r0") / ave_infous,
    source="Susceptible",
    dest="Pre-infectious"
)
model.add_transition_flow(
    name="progression", 
    fractional_rate=1. / Parameter("ave_preinfous"),
    source="Pre-infectious", 
    dest="Infectious"
)
model.add_transition_flow(
    name="recovery", 
    fractional_rate=1. / ave_infous,
    source="Infectious", 
    dest="Immune"
)
parameters = {
    "r0": 2.1,
    "ave_preinfous": 2.,
    "ave_infous": 2.,
    "life_expectancy": 70.,
}
model.run(parameters=parameters, solver="euler")
seir_values = model.get_outputs_df()
assert abs(seir_data - seir_values).max().max() < 1e-5